### Faire tous les imports nécessaires

In [1]:
import pandas as pd
import string
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import re

### Importer les données

In [2]:
# les components
test_components_data = pd.read_csv('./data/test_components.csv')
train_components_data = pd.read_csv('./data/train_components.csv')
validation_components_data = pd.read_csv('./data/validation_components.csv')

# ajouts des 2 colonnes de contexte aux components
test_components_data['context1'] = ''
test_components_data['context2'] = ''
train_components_data['context1'] = ''
train_components_data['context2'] = ''
validation_components_data['context1'] = ''
validation_components_data['context2'] = ''

# les speechs
test_speeches_data = pd.read_csv('./data/test_speeches.csv')
train_speeches_data = pd.read_csv('./data/train_speeches.csv')
validation_speeches_data = pd.read_csv('./data/validation_speeches.csv')

# données de statistiques
size_components_data = len(test_components_data) + len(train_components_data) + len(validation_components_data)
print("Nombre de composants : ", size_components_data)

Nombre de composants :  32988


Petite fonction pour les resultats intermédiaires

In [3]:
# fonction permettant de retourner string permettant un print en couleur sur un pourcentage en python
# plus le pourcentage est proche de 0 plus il sera rouge, plus il est proche de 100 plus il sera vert
# le pourcentage sera calculé en fonction de la taille du dataset et d'un chiffre donné
def printResultColor(size_dataset, number):
    pourcentage = number / size_dataset * 100
    if pourcentage < 1:
        # print vert
        return("\033[92m {0:0.3f}%\033[0m".format(pourcentage))
    elif pourcentage < 5:
        # print orange
        return("\033[93m {0:0.3f}%\033[0m".format(pourcentage))
    else:
        # print rouge
        return("\033[91m {0:0.3f}%\033[0m".format(pourcentage))

### Implémentation de la premiere fonction

Cette fonction doit trouver le Text de tous components dans les Speeches. Une fois chaque Text trouvé, il faut completer le context 1 et 2.

Cette fonction renvoie une liste de tous les identifiants des components qui n'ont pas été trouvés et une liste de tous les composants qui ont été trouvés plusieur fois (multiple speeches).

In [4]:
def find1Component(dataset_component, dataset_speeches):
    not_found = []
    multiple_speech = []
    for index, row in dataset_component.iterrows():
        textToFind = row.Text
        speeches = dataset_speeches[dataset_speeches['Speech'].str.find(textToFind) != -1]
        # verifier que le component est bien dans un speech
        if len(speeches) == 0:
            not_found.append(index)
        elif len(speeches) == 1:
            # trouver le contexte 1 et 2
            # tokenizer le speech à l'aide de nltk
            sentences = sent_tokenize(speeches.iloc[0].Speech)
            # trouver la position de la phrase contenant le component
            position = -1
            for i in range(len(sentences)):
                if sentences[i].find(textToFind.strip()) != -1:
                    position = i
                    break
            if position == -1:
                # le component n'a pas été trouvé dans le speech car il est en plusieurs phrases
                # ici nous allons donc faire un autre traitement pour trouver le contexte 1 et 2
                # dans un premier temps nous allons tokenizer le component en phrases
                component_sentences = sent_tokenize(textToFind)
                # ensuite nous allons chercher la position de la phrase contenant le component
                position = -1
                positions = np.where([sentences[i].find(component_sentences[0].strip()) != -1 for i in range(len(sentences))])
                for i in range(len(positions[0])):
                    if sentences[positions[0][i]+1].find(component_sentences[1].strip()) != -1:
                        position = positions[0][i]
                        break
                try:
                    if(position != -1):
                        # verifier que la suite du component est bien dans la phrase suivante
                        if sentences[position + 1].find(component_sentences[1].strip()) != -1:
                            # trouver le contexte 1
                            if position > 0:
                                dataset_component.at[index, 'context1'] = sentences[position - 1]
                            # ajouter la phrase contenant le component au contexte 1
                            dataset_component.at[index, 'context1'] += sentences[position]
                            dataset_component.at[index, 'context1'] += sentences[position + 1]
                            if position < len(sentences) - 2:
                                # ajouter la phrase qui suit le component au contexte 1
                                dataset_component.at[index, 'context1'] += sentences[position + 2]
                            
                            # mettre tout le speech dans le contexte 2
                            dataset_component.at[index, 'context2'] = speeches.iloc[0].Speech
                        else:
                            print("La suite du component {} n'a pas été trouvés ?".format(index))
                    else:
                        print("Le component {} n'a pas été trouvé en cherchant sur les phrases".format(index))
                except:
                    print("Something went wrong with the component: {}".format(textToFind))
            else:
                # trouver le contexte 1
                if position > 0:
                    dataset_component.at[index, 'context1'] = sentences[position - 1]
                # ajouter la phrase contenant le component au contexte 1
                dataset_component.at[index, 'context1'] += sentences[position]
                # ajouter la phrase qui suit le component au contexte 1
                if position < len(sentences) - 1:
                    dataset_component.at[index, 'context1'] += sentences[position + 1]
                
                # mettre tout le speech dans le contexte 2
                dataset_component.at[index, 'context2'] = speeches.iloc[0].Speech
        else:
            # plusieurs speeches contiennent le component
            # il va falloir verifier l'id du speech et de la section pour verifier si c'est le bon
            multiple_speech.append(index)
    return not_found, multiple_speech

In [5]:
# tester la fonction
not_founded_test = []
multiple_founded_test = []
not_founded_test, multiple_founded_test = find1Component(test_components_data, test_speeches_data)
print("Nombre de composants test non trouvés: {} -> {}".format(len(not_founded_test), printResultColor(len(test_components_data), len(not_founded_test))))
print("Nombre de composants test trouvés dans plusieurs discours: {} -> {}\n".format(len(multiple_founded_test), printResultColor(len(test_components_data), len(multiple_founded_test))))

not_founded_train = []
multiple_founded_train = []
not_founded_train, multiple_founded_train = find1Component(train_components_data, train_speeches_data)
print("Nombre de composants train non trouvés: {} -> {}".format(len(not_founded_train), printResultColor(len(train_components_data), len(not_founded_train))))
print("Nombre de composants train trouvés dans plusieurs discours: {} -> {}\n".format(len(multiple_founded_train), printResultColor(len(train_components_data), len(multiple_founded_train))))

not_founded_validation = []
multiple_founded_validation = []
not_founded_validation, multiple_founded_validation = find1Component(validation_components_data, validation_speeches_data)
print("Nombre de composants validation non trouvés: {} -> {}".format(len(not_founded_validation), printResultColor(len(validation_components_data), len(not_founded_validation))))
print("Nombre de composants validation trouvés dans plusieurs discours: {} -> {}\n".format(len(multiple_founded_validation), printResultColor(len(validation_components_data), len(multiple_founded_validation))))


Nombre de composants test non trouvés: 44 ->  0.463%
Nombre de composants test trouvés dans plusieurs discours: 1030 ->  10.850%

Nombre de composants train non trouvés: 106 ->  0.621%
Nombre de composants train trouvés dans plusieurs discours: 777 ->  4.552%

Nombre de composants validation non trouvés: 36 ->  0.560%
Nombre de composants validation trouvés dans plusieurs discours: 144 ->  2.241%



In [6]:
# en affichant le text du premier component non trouvé, on se rend compte que tout le component est juste mais qu'il manque le point entre les deux phrases (d'où le fait que l'on ne le trouve pas)
component_text = test_components_data.iloc[not_founded_test[0]].Text
print(component_text)
# print le speech test SectionID 9 SpeechID 1 de KENNEDY (id 9)
speech_kennedy = test_speeches_data.iloc[6].Speech
# tokenize le speech en phrases pour faciliter l'affichage
sentences = sent_tokenize(speech_kennedy)
# afficher les phrases 25 et 26
print(sentences[25:27])

# Cet exemple montre la pertinence de la fonction suivante

I believe it my responsibility as the leader of the Democratic party in 1960 to try to warn the American people that in this crucial time we can no longer afford to stand still We can no longer afford to be second best
['I believe it my responsibility as the leader of the Democratic party in 1960 to try to warn the American people that in this crucial time we can no longer afford to stand still.', 'We can no longer afford to be second best.']


### Implémentation de la deuxième fonction

Cette fonction doit trouver le Text de tous les components qui sont en plusieurs phrases et qui n'ont pas été trouvés précédements, avec ou sans la ponctuation. Le resultat renvoyé sera similaire à celui de la fonction précédente.

In [7]:
# def find2Components(idsComponents, dataset_component, dataset_speeches):
#     not_found = []
#     # pour chaque id de component
#     for idComponent in idsComponents:
#         # get le text du component
#         textToFind = re.sub(r'[\.!\?]?( -)?', '', dataset_component.iloc[idComponent].Text)
#         # find the text in the speeches, but this time we remove the ponctuations of the speeches using replace function of pandas
#         speeches = dataset_speeches[dataset_speeches['Speech'].str.replace(r'[\.!\?]?( -)?', '', regex=True).str.find(textToFind) != -1]
#         if(len(speeches) == 1):
#             if(idComponent == 187):
#                 print("ici")
#             speeches = speeches.iloc[0]
#             # si on a trouvé le component dans un seul speech
#             # tokenizer le component en faisant un split sur les espaces
#             words_component = dataset_component.iloc[idComponent].Text.split(" ")
#             full_component = ""
#             for word in words_component:
#                 if(speeches.Speech.find(full_component) != -1):
#                     # si le component est dans le texte
#                     # on ajoute le mot suivant au component
#                     if(len(full_component) > 0):
#                         full_component += " "+word
#                     else:
#                         full_component = word
#                 else:
#                     # il faut enlever le dernier mot du component
#                     # car il manque surement une ponctuation
#                     last = full_component[full_component.rfind(" "):]
#                     # remove the last word from the component
#                     full_component = full_component[:full_component.rfind(" ")]
#                     letter = speeches.Speech[speeches.Speech.find(full_component)+len(full_component)]
#                     if(speeches.Speech.find(full_component+letter+last) != -1):
#                         full_component += letter+last+" "+word
#                     elif(speeches.Speech.find(full_component+" "+letter+last) != -1):
#                         full_component += " "+letter+last+" "+word
#             # print the component before and after the correction in a file to see the difference
#             with open("components.txt", "a") as file:
#                 file.write(dataset_component.iloc[idComponent].Text+"\n"+full_component+"\n\n")
                
#         else:
#             not_found.append(idComponent)
#     print("Nombre de composants trouvés {} sur {}".format(len(idsComponents) - len(not_found), len(idsComponents)))
#     return not_found

In [8]:
def find3Components(idsComponents, dataset_component, dataset_speeches):
    not_found = []
    # pour chaque id de component
    for idComponent in idsComponents:
        print(idComponent)
        # split le component en mots using nltk
        words_component = word_tokenize(dataset_component.iloc[idComponent].Text)
        if idComponent == 56:
            # speeches is a list of rows of speeches that contain the component
            speeches = []
            for n in range(1, len(words_component)):
                # concat les n premiers mots du component
                start_component = " ".join(words_component[0:n])
                end_component = " ".join(words_component[n:])
                # get all the speeches that contain the start component and the end component
                speeches_intermed = dataset_speeches[dataset_speeches['Speech'].str.find(start_component) != -1]
                speeches_intermed = speeches_intermed[speeches_intermed['Speech'].str.find(end_component) != -1]
                if(len(speeches_intermed) > 0):
                    # append each speech that contains the component
                    speeches.append(speeches_intermed)
            print(speeches)
            if(len(speeches) > 0):
                if(len(speeches) == 1):
                    # on l'a surrement trouvé
                    print(speeches[0])
                else:
                    # plusieurs résultats, cas particulier à traiter
                    not_found.append(idComponent)
not_fnd = find3Components(not_founded_test, test_components_data, test_speeches_data)


56


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
# 34

Text index 603 of test component, dataset (id 603, Speech: <font color="red">0</font>, Section: 8, Speaker: <font color="red">Walters</font>):

<font color="orange">the tax program of the Ford administration will bring jobs where people are, and help to revitalize those cities as they can be</font>

The component of the dataset is (id: 62 - speech <font color="green">4</font> section 8 by <font color="green">Ford</font>):

(...) The net result is uh - we think under Carla Hills, who's the chairman of my uh - urban development and uh - neighborhood revitalization program, we will really do a first-class job in helping uh - the communities throughout the country. As a matter of fact, that committee under Secretary Hills released about a seventy-five-page report with specific recommendations so we can do a better job uh - the weeks ahead. And in addition, <font color="orange">the tax program of the Ford administration</font><font color="red">, which provides an incentive for industry to move into our major uh - metropolitan areas, into the inner cities,</font><font color="orange"> will bring jobs where people are, and help to revitalize those cities as they can be</font>.

**In this case, should we patch the Text of the dataset component with the punctuation and the missing part of the sentence or not?**

**The speaker name as well as the section number is false, do we need to find an automatic way to correct these errors?**

Text index 187 of train component:

<font color="orange"> I don't think it was helpful when he suggested that seventeen million people go to bed hungry every night in the United States</font>

The component of the dataset is uncorrected (id: 22 - speech 2 section 11 by Nixon):

I did. And as I pointed out in 1952, (...) I don't think he should say that our prestige is at an all-time low. I think this is very harmful at a time Mr. Khrushchev is here - harmful because it's wrong. <font color="orange">I don't think it was helpful when he suggested</font><font color="red"> - and I'm glad he's corrected this to an extent - </font><font color="orange">that seventeen million people go to bed hungry every night in the United States</font>. Now this just wasn't true. Now, there are people who go to bed hungry in the United States - far less, incidentally, than used to go to bed hungry when we came into power at the end of the Truman Administration. But the thing that is right about the United States, it should be emphasized, (...)


**The component does not take into account all the text of the sentence. Do we must automatically patch it because it is a case that repeats itself quite frequently?**

**Another example would be, if one of our components is between two sentences, can we delimitate as follows:**

<font color="purple">A donut is a type of pastry that is typically round with a hole in the middle. </font><font color="green">It is usually fried and made from a sweet dough that can be flavored with </font><font color="orange">various ingredients such as cinnamon or chocolate. Donuts are often served as a breakfast or dessert</font><font color="green"> food and can be enjoyed plain or with toppings like frosting, sprinkles, or fruit.</font> <font color="blue">In recent years, donuts have become a popular trend in the culinary world, with many bakeries offering unique and creative flavors and designs.</font> Despite their delicious taste, donuts should be consumed in moderation as they are high in sugar and calories.

<font color="orange">Component</font>: various ingredients such as cinnamon or chocolate. Donuts are often served as a breakfast or dessert

<font color="green">Current</font>: It is usually fried and made from a sweet dough that can be flavored with various ingredients such as cinnamon or chocolate. Donuts are often served as a breakfast or dessert food and can be enjoyed plain or with toppings like frosting, sprinkles, or fruit.

<font color="purple">Previous</font>: A donut is a type of pastry that is typically round with a hole in the middle. 

<font color="blue">Next</font>: In recent years, donuts have become a popular trend in the culinary world, with many bakeries offering unique and creative flavors and designs.

### Implémentation de la troisième fonction

Cette fonction va chercher tous les composants qui ont été trouvés dans plusieurs speechs et verifier si il est possible de trouver le speech correspondant à notre composant.

In [ ]:
def find3Components(idsComponents, dataset_component, dataset_speeches):
    # first we need to create an empty array wich will contain the components that we will not find
    not_found = []
    not_found_multiple = []
    # next we need to iterate over each component
    for idComponent in idsComponents:
        # get the component
        component = dataset_component.iloc[idComponent]
        # get the text of the component
        textToFind = component.Text
        # get all the speeches that contain the component and wich have the same SpeechID and SectionID
        speeches = dataset_speeches[dataset_speeches['Speech'].str.find(textToFind) != -1]
        speeches = speeches[speeches['SpeechID'] == component.SpeechID]
        speeches = speeches[speeches['SectionID'] == component.SectionID]
        # if we find the component in one speech
        if(len(speeches) == 1):
            # we find the right speech
            # get the speech text
            speech = speeches.iloc[0].Speech
            sentences = sent_tokenize(speech)
            # trouver la position de la phrase contenant le component
            position = -1
            for i in range(len(sentences)):
                if sentences[i].find(textToFind.strip()) != -1:
                    position = i
                    break
            # si on a trouvé la position de la phrase
            if(position != -1):
                if position > 0:
                    dataset_component.at[idComponent, 'context1'] = sentences[position - 1]
                # ajouter la phrase contenant le component au contexte 1
                dataset_component.at[idComponent, 'context1'] += sentences[position]
                # ajouter la phrase qui suit le component au contexte 1
                if position < len(sentences) - 1:
                    dataset_component.at[idComponent, 'context1'] += sentences[position + 1]
                
                # mettre tout le speech dans le contexte 2
                dataset_component.at[idComponent, 'context2'] = speech
            else:
                # print("position not found for {}".format(idComponent))
                not_found.append(idComponent)
            
        elif(len(speeches) > 1):
            # this technique is not enough to find the component
            not_found_multiple.append(idComponent)
        else:
            # we simply not found the component with this technique, mabe the component SectionID or SpeechID is wrong
            not_found.append(idComponent)
    return not_found, not_found_multiple

In [ ]:
not_found_multi_test, not_found_in_one_test = find3Components(multiple_founded_test, test_components_data, test_speeches_data)
print("Nombre de composants test multiple non trouvés: {} sur les {} -> {}".format(len(not_found_multi_test), len(multiple_founded_test), printResultColor(len(multiple_founded_test), len(not_found_multi_test))))
print("Nombre de composants test multiple non trouvés dans qu'un seul speech: {} sur les {} -> {}\n".format(len(not_found_in_one_test), len(multiple_founded_test), printResultColor(len(multiple_founded_test), len(not_found_in_one_test))))

not_found_multi_train, not_found_in_one_train = find3Components(multiple_founded_train, train_components_data, train_speeches_data)
print("Nombre de composants train multiple non trouvés: {} sur les {} -> {}".format(len(not_found_multi_train), len(multiple_founded_train), printResultColor(len(multiple_founded_train), len(not_found_multi_train))))
print("Nombre de composants train multiple non trouvés dans qu'un seul speech: {} sur les {} -> {}\n".format(len(not_found_in_one_train), len(multiple_founded_train), printResultColor(len(multiple_founded_train), len(not_found_in_one_train))))

not_found_multi_validation, not_found_in_one_validation = find3Components(multiple_founded_validation, validation_components_data, validation_speeches_data)
print("Nombre de composants validation multiple non trouvés: {} sur les {} -> {}".format(len(not_found_multi_validation), len(multiple_founded_validation), printResultColor(len(multiple_founded_validation), len(not_found_multi_validation))))
print("Nombre de composants validation multiple non trouvés dans qu'un seul speech: {} sur les {} -> {}\n".format(len(not_found_in_one_validation), len(multiple_founded_validation), printResultColor(len(multiple_founded_validation), len(not_found_in_one_validation))))

Nombre de composants test multiple non trouvés: 9 sur les 1030 ->  0.874%
Nombre de composants test multiple non trouvés dans qu'un seul speech: 10 sur les 1030 ->  0.971%

Nombre de composants train multiple non trouvés: 10 sur les 777 ->  1.287%
Nombre de composants train multiple non trouvés dans qu'un seul speech: 21 sur les 777 ->  2.703%

Nombre de composants validation multiple non trouvés: 0 sur les 144 ->  0.000%
Nombre de composants validation multiple non trouvés dans qu'un seul speech: 4 sur les 144 ->  2.778%



In [ ]:
# print un élément non trouvé de la liste not_found_multi_train
print(not_found_multi_train[0])
# get les ids des speeches qui contiennent le component
speeches = train_speeches_data[train_speeches_data['Speech'].str.find(train_components_data.iloc[not_found_multi_train[0]].Text) != -1]
print(speeches.index)

7265
Int64Index([1395, 1495], dtype='int64')


In [ ]:
# regex to find 2 line with the same text
# ^([^\n]+)\n\1$



ID 622 et 582 de la liste speech test sont exactement les mêmes sauf leurs Section ID  (21_1992 et 9)

Le premier cas est il un ID + une date ? (21_1992)

Dans les deux cas, pourquoi nous avons des tuples dans notre dataset.

In [ ]:
# get le nombre de component qui n'ont pas de conexte 1 et 2
nb = len(test_components_data[train_components_data['context1'] == ""])
print("Nombre de composants test sans contexte 1 et 2: {}".format(nb))
print("Pourcecentage de composants test sans contexte 1 et 2: {}%".format(nb / len(test_components_data) * 100))

nb_train = len(train_components_data[train_components_data['context1'] == ""])
print("Nombre de composants train sans contexte 1 et 2: {}".format(nb_train))
print("Pourcecentage de composants train sans contexte 1 et 2: {}%".format(nb_train / len(train_components_data) * 100))

nb_validation = len(validation_components_data[validation_components_data['context1'] == ""])
print("Nombre de composants validation sans contexte 1 et 2: {}".format(nb_validation))
print("Pourcecentage de composants validation sans contexte 1 et 2: {}%".format(nb_validation / len(validation_components_data) * 100))


Nombre de composants test sans contexte 1 et 2: 80
Pourcecentage de composants test sans contexte 1 et 2: 0.8427262193194985%
Nombre de composants train sans contexte 1 et 2: 137
Pourcecentage de composants train sans contexte 1 et 2: 0.8026246411623411%
Nombre de composants validation sans contexte 1 et 2: 40
Pourcecentage de composants validation sans contexte 1 et 2: 0.6224712107065048%


C:\Users\user\AppData\Local\Temp\ipykernel_17948\1113298283.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  nb = len(test_components_data[train_components_data['context1'] == ""])
